In [1]:
from os import path, listdir

import h5py
import numpy as np
from scipy.io import loadmat

from spikelib.utils import check_groups

* /sta/intensity/raw/temp*
* /sta/intensity/temporal/raw/temp* --> attrs['time'], attrs['nsample_before'], attrs['nsample_adfter'], attrs['fps'], attrs['nsamples']
* /sta/intensity/temporal/fit/temp* --> attrs['time']
* /sta/intensity/temporal/fit_params/temp* --> attrs['col_name']
* /sta/intensity/temporal/char/temp* --> attrs['col_name'] peak1, peak2, amp1, amp2, zeroxross, bi, 
* /sta/intensity/spatial/char/temp* --> attrs['col_name'] angle, a,b,x,y,snr,frame
* /sta/intensity/valid/temp* 



In [22]:
def save_fitres_to_hdf5(source_folder, raw_sta_group, output_group):
    units = listdir(source_folder)
    units = [kunit for kunit in units if path.isdir(source_folder+kunit)]
    file = 'fit_var.mat'

    for kunit in units:
        kfile = '{}{}/{}'.format(source_folder, kunit, file)
        if path.isfile(kfile):
            fitres = loadmat(kfile)['fitresult'].flatten()
            amp, deg, a, b, x0, y0, z, frame = fitres
            x0 = x0-1
            y0 = y0-1
            frame = frame-1 

            sta_data = raw_sta_group[kunit][...]
            temp_profile = sta_data[:,int(round(y0)),int(round(x0))]
            var_resp = np.var(temp_profile)
            var_noise = np.var(sta_data, axis=2).mean()
            snr = var_resp/var_noise       
        else:
            [deg, a, b, x0, y0, snr, frame] = np.zeros(7)  
        
        if kunit in output_group:           
            output_group[kunit][...] = np.array([deg, a, b, x0, y0, snr, frame])
            output_group.attrs['col_name'] = u'angle,semia,semib,x0,y0,snr,frame_max'
        else:
            output_group.create_dataset(kunit, data=np.array([deg, a, b, x0, y0, snr, frame]),
                                        dtype=np.float, compression="gzip")
            output_group.attrs['col_name'] = u'angle,semia,semib,x0,y0,snr,frame_max'

def save_fitrest_mat_to_hdf5(source_file, raw_sta_group, output_group):
    """Save RF fit result from a .mat to hdf5.
    
    The .mat file has a cell array with all fitresulta and the keys for each 
    cell.

    """
    
    
    fit_rest = loadmat(source_file)

    for karr, key in zip(fit_rest['fit_array'][0], fit_rest['keys'][0]):
        fit_array = karr[0]
        kunit = key[0]
        if fit_array.any():
            amp, deg, a, b, x0, y0, z, frame = fit_array
            x0 = x0-1
            y0 = y0-1
            frame = frame-1 

            sta_data = raw_sta_group[kunit][...]
            temp_profile = sta_data[:,int(round(y0)),int(round(x0))]
            var_resp = np.var(temp_profile)
            var_noise = np.var(sta_data, axis=2).mean()
            snr = var_resp/var_noise       
        else:
            [deg, a, b, x0, y0, snr, frame] = np.zeros(7)  
        
        if kunit in output_group:           
            output_group[kunit][...] = np.array([deg, a, b, x0, y0, snr, frame])
        else:
            output_group.create_dataset(kunit, data=np.array([deg, a, b, x0, y0, snr, frame]),
                                        dtype=np.float, compression="gzip")
    output_group.attrs['col_name'] = u'angle,semia,semib,x0,y0,snr,frame_max'
    

def save_temp_raw_to_hdf5(output_group, spatial_char_group, raw_group):
    
    for key in spatial_char_group:
        (angle, a, b, x0, y0, snr, frame) = spatial_char_group[key][...]
        if key in output_group:
            output_group[key][...] = raw_group[key][:,y0,x0]
        else:
            output_group.create_dataset(key, data=raw_group[key][:,y0,x0], dtype=np.float, compression="gzip")

    output_group.attrs['fps'] = raw_group.attrs['fps']
    output_group.attrs['time'] = raw_group.attrs['time']
    output_group.attrs['nsample_before'] = raw_group.attrs['nsamples_before']
    output_group.attrs['nsample_after'] = raw_group.attrs['nsamples_after']
    output_group.attrs['nsamples'] = raw_group.attrs['nsamples']

## Save fitres to hdf5

In [20]:
fanalisis = '../data/MR-0092t2_analysis_of_protocols_150um_merge.hdf5'

In [26]:
intensities = ['nd2-255', 'nd3-255', 'nd4-255', 'nd5-255']
for intensity in intensities:
    source_folder = '/home/cesar/exp/MR-0092t2/{}/'.format(intensity)

    with h5py.File(fanalisis , 'r+') as panalisis:
        raw_sta_group = '/sta/{}/raw/'.format(intensity)
        output_group = '/sta/{}/spatial/char/'.format(intensity)
        if raw_sta_group in panalisis:
            check_groups(panalisis, [output_group])
            save_fitres_to_hdf5(source_folder,
                                panalisis[raw_sta_group],
                                panalisis[output_group],
                               )

OSError: [Errno 2] No such file or directory: '/home/cesar/exp/MR-0092t2/nd2-255/'

In [23]:
intensities = ['nd2-255', 'nd3-255', 'nd4-255', 'nd5-255']
for intensity in intensities:
    source_file = '/home/cesar/Downloads/sta/ajuste_{}.mat'.format(intensity)

    with h5py.File(fanalisis , 'r+') as panalisis:
        raw_sta_group = '/sta/{}/raw/'.format(intensity)
        output_group = '/sta/{}/spatial/char/'.format(intensity)
        if raw_sta_group in panalisis:
            check_groups(panalisis, [output_group])
            save_fitrest_mat_to_hdf5(source_file,
                                     panalisis[raw_sta_group],
                                     panalisis[output_group],
                                    )

## Save raw temporal integration 

In [25]:
fanalisis = '../data/MR-0092t2_analysis_of_protocols.hdf5'

intensities = ['nd2-255', 'nd3-255', 'nd4-255', 'nd5-255']
for intensity in intensities:
    with h5py.File(fanalisis , 'r+') as panalisis:
        raw_sta_group = '/sta/{}/raw/'.format(intensity)
        spatial_char_group = '/sta/{}/spatial/char/'.format(intensity)
        temp_raw_group = '/sta/{}/temporal/raw/'.format(intensity)

        if raw_sta_group in panalisis:
            check_groups(panalisis, [temp_raw_group])
            save_temp_raw_to_hdf5(panalisis[temp_raw_group],
                                  panalisis[spatial_char_group],
                                  panalisis[raw_sta_group],
                                  )
        else:
            print('{} doesnt exist in {}'.format(raw_sta_group, panalisis))

[u'fps', u'nsamples_before', u'nsamples_after', u'nsamples', u'time']
[u'fps', u'nsamples_before', u'nsamples_after', u'nsamples', u'time']
[u'fps', u'nsamples_before', u'nsamples_after', u'nsamples', u'time']
[u'fps', u'nsamples_before', u'nsamples_after', u'nsamples', u'time']


* /flash/intensity/psth/temp*
* /flash/intensity/est_psth/temp* --> attrs[time], attrs[bouns]
* /flash/intensity/char/temp* --> attrs[col_name],